In [1]:
import random
import time

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

from imblearn.under_sampling import RandomUnderSampler

In [3]:
filepath = "training(1).csv" 
data = pd.read_csv(filepath,',',error_bad_lines=False)
data = data.dropna()
print(len(data))

669423


In [4]:
vectorizer = TfidfVectorizer(tokenizer=list)
X = vectorizer.fit_transform(data.password.values)
y = data.strength

In [164]:
rus = RandomUnderSampler(sampling_strategy='majority', replacement=False, random_state=42)
X_res, y_res = rus.fit_resample(X, y)
np.unique(y_res, return_counts=True)

(array([0, 1, 2]), array([89661, 83113, 83113]))

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)  #splitting
X_val, _, y_val, _  = train_test_split(X_train, y_train, train_size=10**4, random_state=42)  #splitting

In [173]:
clf = SVC()
parameters = {'C': [0.1, 1, 10]}
gcv = GridSearchCV(clf, parameters, scoring='f1_macro', refit=False, cv=5, return_train_score=True)
get_ipython().run_line_magic('time', 'gcv.fit(X_val, y_val)')
print(gcv.best_params_, gcv.best_score_)

CPU times: user 1min 36s, sys: 12.1 ms, total: 1min 37s
Wall time: 1min 37s
{'C': 1} 0.8057680129020733


In [174]:
clf = RandomForestClassifier(random_state=42)
parameters = {'max_depth': [2, 8, 32, 128, None]}
gcv = GridSearchCV(clf, parameters, scoring='f1_macro', refit='f1_macro', cv=5, return_train_score=True)
get_ipython().run_line_magic('time', 'gcv.fit(X_val, y_val)')
print(gcv.best_params_, gcv.best_score_)

CPU times: user 46 s, sys: 7.64 ms, total: 46 s
Wall time: 46.1 s
{'max_depth': 32} 0.9030542153586115


In [176]:
clf = HistGradientBoostingClassifier(random_state=42)
parameters = {'max_depth': [2, 8, 32, 128, None]}
gcv = GridSearchCV(clf, parameters, scoring='f1_macro', refit='f1_wf1_macroeighted', cv=5, return_train_score=True)
get_ipython().run_line_magic('time', 'gcv.fit(X_val.toarray(), y_val)')
print(gcv.best_params_, gcv.best_score_)

CPU times: user 18min 2s, sys: 20.2 s, total: 18min 23s
Wall time: 1min 12s
{'max_depth': 32} 0.9456354905962533


In [178]:
model = HistGradientBoostingClassifier(random_state=42, max_depth=32)
model.fit(X_train.toarray(), y_train)
predictions = model.predict(X_test.toarray())

In [183]:
accuracy = metrics.accuracy_score(y_test, predictions)
f1_score = metrics.f1_score(y_test, predictions,average='macro')
precision = metrics.precision_score(y_test, predictions,average='macro')
recall = metrics.recall_score(y_test, predictions,average='macro')
accuracy, f1_score, precision, recall

(0.9711985618820587,
 0.9709798297247095,
 0.9713967162151262,
 0.9707985270967238)